## Classification

To run this script, you need the following files found in the /data directory:
- "final_labels_SG1.xlsx"
- "final_labels_SG2.xlsx"

## Imports and set-up

In [11]:
# !pip install transformers
!pip install sentencepiece

In [12]:
!pwd

/home/TableSense/largedisk/wanrong/Curation-Modeling/tools/Neural-Media-Bias-Detection-Using-Distant-Supervision-With-BABE


In [13]:
import sys
import os
import time
import re
import random
from typing import Dict, List, Optional, Union
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import tensorflow as tf
from transformers import BertTokenizer, BertConfig, TFBertForSequenceClassification
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from transformers import ElectraTokenizer, TFElectraForSequenceClassification
from transformers import XLNetTokenizer, TFXLNetForSequenceClassification
from transformers import LongformerTokenizer, TFLongformerForSequenceClassification

In [14]:
# set seed, TF uses python ramdom and numpy library, so these must also be fixed
tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)
os.environ['PYTHONHASHSEED']=str(0)
os.environ['TF_DETERMINISTIC_OPS'] = '0'

In [15]:
# see if hardware accelerator available
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [16]:
# tf.test.gpu_device_name()

If GPUs are available, tensorflow will give priority to it automatically and computations will be performed on the GPU as default. That behavior can be changed by assigning a task explicitly to a device. Example:

```
with tf.device('/CPU:0'):
```



## Preprocessing

In [17]:
PATH_sg1 = "data/final_labels_SG1.xlsx"
PATH_sg2 = "data/final_labels_SG2.xlsx"
df_sg1 = pd.read_excel(PATH_sg1, engine='openpyxl')
df_sg2 = pd.read_excel(PATH_sg2, engine='openpyxl')
df_sg1.rename(columns={'text': 'sentence', 'label_bias': 'Label_bias'}, inplace=True)
df_sg2.rename(columns={'text': 'sentence', 'label_bias': 'Label_bias'}, inplace=True)

# binarize classification problem
df_sg1 = df_sg1[df_sg1['Label_bias']!='No agreement']
df_sg1 = df_sg1[df_sg1['Label_bias'].isna()==False]
df_sg1.replace(to_replace='Biased', value=1, inplace=True)
df_sg1.replace(to_replace='Non-biased', value=0, inplace=True)
df_sg1["type"] = df_sg1["type"].map({"left": 0, "center": 1, "right": 2})

df_sg2 = df_sg2[df_sg2['Label_bias']!='No agreement']
df_sg2 = df_sg2[df_sg2['Label_bias'].isna()==False]
df_sg2.replace(to_replace='Biased', value=1, inplace=True)
df_sg2.replace(to_replace='Non-biased', value=0, inplace=True)
df_sg2["type"] = df_sg2["type"].map({"left": 0, "center": 1, "right": 2})
df_sg2 = df_sg2[df_sg2['type'].isna()==False]
df_sg2.head()


,sentence,news_link,outlet,topic,type,Label_bias,label_opinion,biased_words
0,"""Orange Is the New Black"" star Yael Stone is r...",https://www.foxnews.com/entertainment/australi...,Fox News,environment,2.0,0,Entirely factual,[]
1,"""We have one beautiful law,"" Trump recently sa...",https://www.alternet.org/2020/06/law-and-order...,Alternet,gun control,0.0,1,Somewhat factual but also opinionated,"['bizarre', 'characteristically']"
2,"...immigrants as criminals and eugenics, all o...",https://www.nbcnews.com/news/latino/after-step...,MSNBC,white-nationalism,0.0,1,Expresses writer’s opinion,"['criminals', 'fringe', 'extreme']"
3,...we sounded the alarm in the early months of...,https://www.alternet.org/2019/07/fox-news-has-...,Alternet,white-nationalism,0.0,1,Somewhat factual but also opinionated,[]
9,‘A new low’: Washington Post media critic blow...,https://www.alternet.org/2019/08/a-new-low-was...,Alternet,white-nationalism,0.0,1,Expresses writer’s opinion,"['blows', 'up', 'absurd', 'lies', 'nationalism..."


In [18]:
# Stratified k-Fold instance
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

The rest of the preprocessing needs to be performed inside the folds as a) encoder layers shouldn't be allowed to see whole data to construct the lookups and b) indexing with skfold is not possible when data is in tensorflow format.

In [24]:
# helper functions called in skfold loop

def pd_to_tf(df):
    """convert a pandas dataframe into a tensorflow dataset"""
    target = df.pop('type')
    # target = df.pop('Label_bias')
    sentence = df.pop('sentence')
    return tf.data.Dataset.from_tensor_slices((sentence.values, target.values))

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

def tokenize(df):
    """convert a pandas dataframe into a tensorflow dataset and run hugging face's tokenizer on data"""
    target = df.pop('type')
    # target = df.pop('Label_bias')
    sentence = df.pop('sentence')

    train_encodings = tokenizer(
                        sentence.tolist(),                      
                        add_special_tokens = True, # add [CLS], [SEP]
                        truncation = True, # cut off at max length of the text that can go to BERT
                        padding = True, # add [PAD] tokens
                        return_attention_mask = True, # add attention mask to not focus on pad tokens
              )
    
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(train_encodings), 
         target.tolist()))
    return dataset

## Attention-based models


In [25]:
def run_model_5fold(df_train, model_name, freeze_encoder=True, pretrained=False, plot=False):
  """"freeze flags whether encoder layer should be frozen to not destroy transfer learning. Only set to false when enough data is provided"""

  # these variables will be needed for skfold to select indices
  Y = df_train['type']
  # Y = df_train['Label_bias']
  X = df_train['sentence']

  # hyperparams
  BUFFER_SIZE = 10000
  BATCH_SIZE = 32
  k = 1

  val_loss = []
  val_acc = []
  val_prec = []
  val_rec = []
  val_f1 = []
  val_f1_micro = []
  val_f1_wmacro = []

  for train_index, val_index in skfold.split(X,Y):
    print('### Start fold {}'.format(k))
    
    # split into train and validation set
    train_dataset = df_train.iloc[train_index]
    val_dataset = df_train.iloc[val_index]

    # prepare data for transformer
    train_dataset = tokenize(train_dataset)
    val_dataset = tokenize(val_dataset)

    # mini-batch it
    train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

    # create new model
    if model_name == 'bert':
      model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
    if model_name == 'distilbert':
      model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)
    elif model_name == 'roberta':
      model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
    elif model_name == 'electra':
      model = TFElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator', num_labels=3)
    elif model_name == 'xlnet':
      model = TFXLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=3)
    # save model
    model.save_pretrained('./models/{}'.format(model_name))
    

    if freeze_encoder == True:
      for w in model.get_layer(index=0).weights:
        w._trainable = False

    # compile it
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
    model.compile(optimizer=optimizer, loss=model.compute_loss) 

    # transfer learning
    if pretrained == True:
      model.get_layer(index=0).set_weights(trained_model_layer) # load bias-specific weights
      #model.load_weights('./checkpoints/')
    
    # after 2 epochs without improvement, stop training
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

    # fit it
    history = model.fit(train_dataset, epochs=10, validation_data = val_dataset, callbacks=[callback])
    
    # plot history
    if plot:
      plot_graphs(history,'loss')

    # evaluate
    loss = model.evaluate(val_dataset)
    
    if model_name == 'xlnet':
      yhats = []
      for row in df_train.iloc[val_index]['sentence']:
        input = tokenizer(row, return_tensors="tf")
        output = model(input)
        logits = output.logits.numpy()[0]
        candidates = logits.tolist()
        decision = candidates.index(max(candidates))
        yhats.append(decision)
    else:
      logits = model.predict(val_dataset)  
      yhats = []
      for i in logits[0]:
        # assign class label according to highest logit
        candidates = i.tolist()
        decision = candidates.index(max(candidates))
        yhats.append(decision)
    
    y = []
    for text, label in val_dataset.unbatch():   
      y.append(label.numpy())
    
    val_loss.append(loss)
    val_acc.append(accuracy_score(y, yhats))
    print("Acc", accuracy_score(y, yhats))
    val_prec.append(precision_score(y, yhats, average='macro'))
    val_rec.append(recall_score(y, yhats, average='macro'))
    val_f1.append(f1_score(y, yhats, average='macro'))
    val_f1_micro.append(f1_score(y, yhats, average='micro'))
    val_f1_wmacro.append(f1_score(y, yhats, average='weighted'))

    tf.keras.backend.clear_session()

    model.save_pretrained('./models/{}'.format(model_name))
    k += 1

  return val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro

### BERT

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# without distant signal pretraining
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='bert', freeze_encoder=False, pretrained=False)

# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

### Start fold 1


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10


/home/TableSense/anaconda3/envs/cr4/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:376: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)


39/39 [==============================] - 165s 4s/step - loss: 1.0493 - val_loss: 1.0375
Epoch 2/10
39/39 [==============================] - 144s 4s/step - loss: 1.0074 - val_loss: 0.9707
Epoch 3/10
39/39 [==============================] - 145s 4s/step - loss: 0.7792 - val_loss: 0.9311
Epoch 4/10
10/10 [==============================] - 13s 1s/step - loss: 0.9311
Acc 0.5838709677419355
### Start fold 2


...

### Start fold 5


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10


/home/TableSense/anaconda3/envs/cr4/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:376: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)


39/39 [==============================] - 196s 5s/step - loss: 1.0276 - val_loss: 0.9930
Epoch 2/10
39/39 [==============================] - 175s 4s/step - loss: 0.8936 - val_loss: 0.9320
Epoch 3/10
10/10 [==============================] - 10s 948ms/step - loss: 0.9320
Acc 0.5048543689320388
5-Fold CV Loss: 0.9471697688102723
5-Fold CV Accuracy: 0.5057709573024324
5-Fold CV Precision: 0.48639225003100617
5-Fold CV Recall: 0.48554703903309704
5-Fold CV F1 Score: 0.4651019179824238
5-Fold CV Micro F1 Score: 0.5057709573024324
5-Fold CV Weighted Macro F1 Score: 0.4818895164133871


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# without distant signal pretraining
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='bert', freeze_encoder=False, pretrained=False)

### Start fold 1


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10


/home/TableSense/anaconda3/envs/cr4/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:376: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)


67/67 [==============================] - 327s 5s/step - loss: 0.9914 - val_loss: 0.9097
Epoch 2/10
67/67 [==============================] - 308s 5s/step - loss: 0.7859 - val_loss: 0.8582
Epoch 3/10
17/17 [==============================] - 23s 1s/step - loss: 0.8582
Acc 0.611214953271028
### Start fold 2


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10


/home/TableSense/anaconda3/envs/cr4/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:376: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)


 5/67 [=>............................] - ETA: 4:27 - loss: 1.1435

In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for BERT on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for BERT on SG2
5-Fold CV Loss: 0.8832831859588623
5-Fold CV Accuracy: 0.5899499457453884
5-Fold CV Precision: 0.5962224353722474
5-Fold CV Recall: 0.6007675671746269
5-Fold CV F1 Score: 0.5830446261476042
5-Fold CV Micro F1 Score: 0.5899499457453884
5-Fold CV Weighted Macro F1 Score: 0.5763379436017781


### BERT + distant

In [ ]:
# load model layer weights from pretraining on distant dataset 
# compile model
#transfer_model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
transfer_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
transfer_model.compile(optimizer=optimizer, loss=transfer_model.compute_loss) 

transfer_model.load_weights('./checkpoints/bert_final_checkpoint_news_headlines_USA')
trained_model_layer = transfer_model.get_layer(index=0).get_weights()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# with distant signal pretraining
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='bert', 
                                                                                            freeze_encoder=False, pretrained=True)

In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for BERT + distant on SG1')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for BERT + distant on SG1
5-Fold CV Loss: 0.4784796714782715
5-Fold CV Accuracy: 0.7791283150506451
5-Fold CV Precision: 0.782222153066104
5-Fold CV Recall: 0.7624161073825504
5-Fold CV F1 Score: 0.7690257043162897
5-Fold CV Micro F1 Score: 0.7791283150506451
5-Fold CV Weighted Macro F1 Score: 0.7782424758051683


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# with distant signal pretraining
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='bert', 
                                                                                            freeze_encoder=False, pretrained=True)

In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for BERT + distant on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for BERT + distant on SG2
5-Fold CV Loss: 0.45280778408050537
5-Fold CV Accuracy: 0.8047893380785556
5-Fold CV Precision: 0.8010278449136129
5-Fold CV Recall: 0.8082872928176796
5-Fold CV F1 Score: 0.8027645188619399
5-Fold CV Micro F1 Score: 0.8047893380785558
5-Fold CV Weighted Macro F1 Score: 0.8043577261879854


### DistilBERT

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='distilbert', 
                                                                                            freeze_encoder=False, pretrained=False)

In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for DistilBERT on SG1')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for DistilBERT on SG1
5-Fold CV Loss: 0.5034848332405091
5-Fold CV Accuracy: 0.7603517841381919
5-Fold CV Precision: 0.7851353351520555
5-Fold CV Recall: 0.7006711409395974
5-Fold CV F1 Score: 0.736955546776338
5-Fold CV Micro F1 Score: 0.7603517841381919
5-Fold CV Weighted Macro F1 Score: 0.758224422060177


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='distilbert', 
                                                                                            freeze_encoder=False, pretrained=False)

In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for DistilBERT on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for DistilBERT on SG2
5-Fold CV Loss: 0.47459145188331603
5-Fold CV Accuracy: 0.7783788392741292
5-Fold CV Precision: 0.7914496007137517
5-Fold CV Recall: 0.756353591160221
5-Fold CV F1 Score: 0.768966991077014
5-Fold CV Micro F1 Score: 0.7783788392741293
5-Fold CV Weighted Macro F1 Score: 0.777102422253571


### RoBERTa

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='roberta', 
                                                                                            freeze_encoder=False, pretrained=False)

### Start fold 1


AttributeError: module 'tensorflow.python.training.experimental.mixed_precision' has no attribute 'register_loss_scale_wrapper'

In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for RoBERTa on SG1')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for RoBERTa on SG1
5-Fold CV Loss: 0.4697426736354828
5-Fold CV Accuracy: 0.7784621527339974
5-Fold CV Precision: 0.7825771209819836
5-Fold CV Recall: 0.7758389261744967
5-Fold CV F1 Score: 0.7715627724463581
5-Fold CV Micro F1 Score: 0.7784621527339974
5-Fold CV Weighted Macro F1 Score: 0.7752193988259601


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='roberta', 
                                                                                            freeze_encoder=False, pretrained=False)

In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for RoBERTa on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for RoBERTa on SG2
5-Fold CV Loss: 0.45095362067222594
5-Fold CV Accuracy: 0.8007054811025227
5-Fold CV Precision: 0.7834945430249991
5-Fold CV Recall: 0.8337016574585636
5-Fold CV F1 Score: 0.8042913565026074
5-Fold CV Micro F1 Score: 0.8007054811025227
5-Fold CV Weighted Macro F1 Score: 0.799347878132545


### RoBERTa + distant

In [ ]:
# load model layer weights from pretraining on distant dataset 
# compile model
transfer_model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
transfer_model.compile(optimizer=optimizer, loss=transfer_model.compute_loss) 

transfer_model.load_weights('./checkpoints/roberta_final_checkpoint_news_headlines_USA')
trained_model_layer = transfer_model.get_layer(index=0).get_weights()

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='roberta', 
                                                                                            freeze_encoder=False, pretrained=True)

In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for RoBERTa + distant on SG1')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for RoBERTa + distant on SG1
5-Fold CV Loss: 0.44771517515182496
5-Fold CV Accuracy: 0.7985668053629219
5-Fold CV Precision: 0.7758359785184737
5-Fold CV Recall: 0.832214765100671
5-Fold CV F1 Score: 0.7999256262460596
5-Fold CV Micro F1 Score: 0.7985668053629217
5-Fold CV Weighted Macro F1 Score: 0.7977665899984501


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='roberta', 
                                                                                            freeze_encoder=False, pretrained=True)

### Start fold 1


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 97s 877ms/step - loss: 0.5345 - val_loss: 0.4133
Epoch 2/10
23/23 [==============================] - 5s 234ms/step - loss: 0.4133
### Start fold 2


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 95s 874ms/step - loss: 0.5673 - val_loss: 0.4084
Epoch 2/10
92/92 [==============================] - 77s 841ms/step - loss: 0.3456 - val_loss: 0.4032
Epoch 3/10
23/23 [==============================] - 5s 215ms/step - loss: 0.4032
### Start fold 3


...

Epoch 1/10
92/92 [==============================] - 96s 875ms/step - loss: 0.5386 - val_loss: 0.4585
Epoch 2/10
23/23 [==============================] - 5s 222ms/step - loss: 0.4585
### Start fold 4


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 97s 882ms/step - loss: 0.5422 - val_loss: 0.4699
Epoch 2/10
92/92 [==============================] - 78s 845ms/step - loss: 0.3178 - val_loss: 0.4695
Epoch 3/10
23/23 [==============================] - 5s 233ms/step - loss: 0.4695
### Start fold 5


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 89s 800ms/step - loss: 0.5482 - val_loss: 0.4198
Epoch 2/10
23/23 [==============================] - 6s 263ms/step - loss: 0.4198


In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for RoBERTa + distant on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for RoBERTa + distant on SG2
5-Fold CV Loss: 0.4328740358352661
5-Fold CV Accuracy: 0.8007017738975699
5-Fold CV Precision: 0.8479610779798031
5-Fold CV Recall: 0.72707182320442
5-Fold CV F1 Score: 0.7811997062364262
5-Fold CV Micro F1 Score: 0.8007017738975699
5-Fold CV Weighted Macro F1 Score: 0.7990983671260622


### ELECTRA

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='electra', 
                                                                                            freeze_encoder=False, pretrained=False)

In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for ELECTRA on SG1')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for ELECTRA on SG1
5-Fold CV Loss: 0.5440825581550598
5-Fold CV Accuracy: 0.7448009918883706
5-Fold CV Precision: 0.7614052309887166
5-Fold CV Recall: 0.6966442953020134
5-Fold CV F1 Score: 0.7211321040106528
5-Fold CV Micro F1 Score: 0.7448009918883706
5-Fold CV Weighted Macro F1 Score: 0.7422210680339413


In [ ]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='electra', 
                                                                                            freeze_encoder=False, pretrained=False)

In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for ELECTRA on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for ELECTRA on SG2
5-Fold CV Loss: 0.5140148580074311
5-Fold CV Accuracy: 0.7625935605849968
5-Fold CV Precision: 0.809470758694404
5-Fold CV Recall: 0.6861878453038674
5-Fold CV F1 Score: 0.7379172614803242
5-Fold CV Micro F1 Score: 0.7625935605849969
5-Fold CV Weighted Macro F1 Score: 0.7597981280791523


### XLNET

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg1, model_name='xlnet', 
                                                                                            freeze_encoder=False, pretrained=False)

In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for XLNET on SG1')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for XLNET on SG1
5-Fold CV Loss: 0.5028677642345428
5-Fold CV Accuracy: 0.7616525868953054
5-Fold CV Precision: 0.7908365542157949
5-Fold CV Recall: 0.6939597315436242
5-Fold CV F1 Score: 0.7374808169992624
5-Fold CV Micro F1 Score: 0.7616525868953054
5-Fold CV Weighted Macro F1 Score: 0.7600315195814671


In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
val_loss, val_acc, val_prec, val_rec, val_f1, val_f1_micro, val_f1_wmacro = run_model_5fold(df_sg2, model_name='xlnet', 
                                                                                            freeze_encoder=False, pretrained=False)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### Start fold 1


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary', 'logits_proj']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 120s 1s/step - loss: 0.5646 - val_loss: 0.4400
Epoch 2/10
23/23 [==============================] - 6s 274ms/step - loss: 0.4400
### Start fold 2


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary', 'logits_proj']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch 1/10
92/92 [==============================] - 120s 1s/step - loss: 0.6138 - val_loss: 0.4555
Epoch 2/10
92/92 [==============================] - 102s 1s/step - loss: 0.4001 - val_loss: 0.4433
Epoch 3/10
23/23 [==============================] - 6s 283ms/step - loss: 0.4433
### Start fold 4


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary', 'logits_proj']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 121s 1s/step - loss: 0.5813 - val_loss: 0.4738
Epoch 2/10
23/23 [==============================] - 7s 324ms/step - loss: 0.4738
### Start fold 5


Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary', 'logits_proj']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
92/92 [==============================] - 113s 1s/step - loss: 0.5735 - val_loss: 0.4463
Epoch 2/10
23/23 [==============================] - 8s 350ms/step - loss: 0.4463


In [ ]:
# inspect metrics
loss_cv = np.mean(val_loss)
acc_cv = np.mean(val_acc)
prec_cv = np.mean(val_prec)
rec_cv = np.mean(val_rec)
f1_cv = np.mean(val_f1)
f1_micro_cv = np.mean(val_f1_micro)
f1_wmacro_cv = np.mean(val_f1_wmacro)

print('Results for XLNET on SG2')
print('5-Fold CV Loss: {}'.format(loss_cv))
print('5-Fold CV Accuracy: {}'.format(acc_cv))
print('5-Fold CV Precision: {}'.format(prec_cv))
print('5-Fold CV Recall: {}'.format(rec_cv))
print('5-Fold CV F1 Score: {}'.format(f1_cv))
print('5-Fold CV Micro F1 Score: {}'.format(f1_micro_cv))
print('5-Fold CV Weighted Macro F1 Score: {}'.format(f1_wmacro_cv))

Results for XLNET on SG2
5-Fold CV Loss: 0.4420862317085266
5-Fold CV Accuracy: 0.7974357263341304
5-Fold CV Precision: 0.8162492174406484
5-Fold CV Recall: 0.76353591160221
5-Fold CV F1 Score: 0.7870615420108473
5-Fold CV Micro F1 Score: 0.7974357263341304
5-Fold CV Weighted Macro F1 Score: 0.7967053530729449
